In [1]:
''' 
boto is aws software development kit, allows to create,
delete update aws resources using python script
'''

' \nboto is aws software development kit, allows to create,\ndelete update aws resources using python script\n'

In [8]:
import boto3
import os
import numpy as np

In [3]:
os.environ['AWS_DEFAULT_REGION'] = 'eu-west-2'
myS3 = boto3.resource('s3')
results = myS3.create_bucket(
                             Bucket='yahoo-forecast',
                             CreateBucketConfiguration={'LocationConstraint': os.environ['AWS_DEFAULT_REGION']}
                            )

BucketAlreadyOwnedByYou: An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.

In [9]:
'''
Create train and val csv
'''
!pip install yfinance


In [58]:
import pandas as pd
from datetime import datetime
import yfinance as yf


start_date = datetime(2019,1,1)
end_date = datetime(2020,1,1)

df = yf.download("AAPL", start_date, end_date)

df.reset_index(inplace=True)
df.head()

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,38.722500,39.712502,38.557499,39.480000,37.893330,148158800
1,2019-01-03,35.994999,36.430000,35.500000,35.547501,34.118893,365248800
2,2019-01-04,36.132500,37.137501,35.950001,37.064999,35.575386,234428400
3,2019-01-07,37.174999,37.207500,36.474998,36.982498,35.496212,219111200
4,2019-01-08,37.389999,37.955002,37.130001,37.687500,36.172871,164101200


In [59]:
df.drop(axis = 1, columns = ['Adj Close', "Date"], inplace= True)

In [60]:
df_features = df.iloc[:-1,:]
df_features

,Open,High,Low,Close,Volume
0,38.722500,39.712502,38.557499,39.480000,148158800
1,35.994999,36.430000,35.500000,35.547501,365248800
2,36.132500,37.137501,35.950001,37.064999,234428400
3,37.174999,37.207500,36.474998,36.982498,219111200
4,37.389999,37.955002,37.130001,37.687500,164101200
...,...,...,...,...,...
246,70.132500,71.062500,70.092499,71.000000,98572000
247,71.172501,71.222504,70.730003,71.067497,48478800
248,71.205002,72.495003,71.175003,72.477501,93121200
249,72.779999,73.492500,72.029999,72.449997,146266000


In [61]:
df_target = df.iloc[1:,0].rename("targets")
df_target

1      35.994999
2      36.132500
3      37.174999
4      37.389999
5      37.822498
         ...    
247    71.172501
248    71.205002
249    72.779999
250    72.364998
251    72.482498
Name: targets, Length: 251, dtype: float64

In [62]:
df_features['target'] = list(df_target)
first_column = df_features.pop('target')
df_features.insert(0, 'target', first_column)
df_final = df_features
df_final

/tmp/ipykernel_3688/2666056106.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features['target'] = list(df_target)


,target,Open,High,Low,Close,Volume
0,35.994999,38.722500,39.712502,38.557499,39.480000,148158800
1,36.132500,35.994999,36.430000,35.500000,35.547501,365248800
2,37.174999,36.132500,37.137501,35.950001,37.064999,234428400
3,37.389999,37.174999,37.207500,36.474998,36.982498,219111200
4,37.822498,37.389999,37.955002,37.130001,37.687500,164101200
...,...,...,...,...,...,...
246,71.172501,70.132500,71.062500,70.092499,71.000000,98572000
247,71.205002,71.172501,71.222504,70.730003,71.067497,48478800
248,72.779999,71.205002,72.495003,71.175003,72.477501,93121200
249,72.364998,72.779999,73.492500,72.029999,72.449997,146266000


In [63]:
import numpy as np
df_randomized = df_final.sample(frac=1, random_state =123)
df_randomized


,target,Open,High,Low,Close,Volume
206,61.855000,60.790001,61.682499,60.720001,61.645000,73477200
188,55.764999,56.267502,57.055000,56.049999,56.147499,139223200
247,71.205002,71.172501,71.222504,70.730003,71.067497,48478800
31,42.427502,42.812500,42.924999,42.437500,42.605000,98507200
241,69.892502,69.250000,70.197502,69.245003,69.964996,128186000
...,...,...,...,...,...,...
98,45.049999,44.950001,45.134998,44.452499,44.915001,146118800
220,65.919998,65.937500,66.220001,65.525002,65.660004,89182800
66,50.080002,49.105000,50.057499,49.084999,50.025002,103526800
126,50.837502,50.820000,51.110001,50.672501,51.102501,45448000


In [64]:
train, test = np.split(df_randomized, [int(0.8*len(df_randomized))])

In [65]:
import os
bucket_name = 'yahoo-forecast'
prefix = 'xgboost-as-a-built-in-algo'

train_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name, prefix, 'train', 'train.csv')

test_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name, prefix, 'test', 'test.csv')

print(train_csv_path)
print(test_csv_path)



s3://yahoo-forecast/xgboost-as-a-built-in-algo/train/train.csv
s3://yahoo-forecast/xgboost-as-a-built-in-algo/test/test.csv


In [66]:
train.to_csv(train_csv_path, index= False, header=False)

In [67]:
test.to_csv(test_csv_path, index= False, header=False)

In [68]:
'''
Build Xgboost
'''
import sagemaker
import boto3
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput

# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"reg:squarederror",
        "num_round":"50"}

# set an output path where the trained model will be saved
bucket_name = 'yahoo-forecast'
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/{}/output'.format(bucket_name, prefix, 'abalone-xgb-built-in-algo')

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "1.7-1")
display(xgboost_container)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


'764974769150.dkr.ecr.eu-west-2.amazonaws.com/sagemaker-xgboost:1.7-1'

In [72]:

# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances= True)

# define the data type and paths to the training and validation datasets
content_type = "csv"
train_input = TrainingInput("s3://{}/{}/{}/".format(bucket_name, prefix, 'train'), content_type=content_type)
test_input = TrainingInput("s3://{}/{}/{}/".format(bucket_name, prefix, 'test'), content_type=content_type)

# execute the XGBoost training job
estimator.fit({'train': train_input, 'validation': test_input})

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-12-11-00-35-54-283


ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: Invalid MaxWaitTimeInSeconds. It must be present and be greater than or equal to MaxRuntimeInSeconds

In [ ]:
'''
deploy trained xgboost
if delpoyment 
    -within sagemaker -> Serialization by user
    -Outside sagemaker -> Serialization by endpoint
    
Method to invoke endpoint
    -APi-single prediction
    -s3 bucket batch prediction


Data type based on method, to invoke endpoint 
-json
-s3bucket

'''

In [ ]:
'''
when we deploy the model, things to be concerned with
-initial instance count: number of instances to deploy the model to
-instance_type: type of instance you want to deploy the model
'''

In [70]:
from sagemaker.serializers import CSVSerializer

In [71]:
xgb_predictor = estimator.deploy(initial_instance_count=1,
                                 instance_type='ml.m5.2xlarge', 
                                 serializer= CSVSerializer())

ValueError: Estimator is not associated with a training job

In [54]:
xgb_predictor.endpoint_name

'sagemaker-xgboost-2023-12-11-00-25-27-721'

In [55]:
# inference
start_date = datetime(2022,12,12)
end_date = datetime(2023,1,1)

df = yf.download("AAPL", start_date, end_date)

df.reset_index(inplace=True)
df.head()

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-12-12,142.699997,144.500000,141.059998,144.490005,143.686859,70462700
1,2022-12-13,149.500000,149.970001,144.240005,145.470001,144.661438,93886200
2,2022-12-14,145.350006,146.660004,141.160004,143.210007,142.413986,82291200
3,2022-12-15,141.110001,141.800003,136.029999,136.500000,135.741257,98931900
4,2022-12-16,136.690002,137.649994,133.729996,134.509995,133.762314,160156900


In [56]:
df.drop(axis = 1, columns = ['Adj Close', "Date"], inplace= True)
df_array = df.values
df_array

array([[1.42699997e+02, 1.44500000e+02, 1.41059998e+02, 1.44490005e+02,
        7.04627000e+07],
       [1.49500000e+02, 1.49970001e+02, 1.44240005e+02, 1.45470001e+02,
        9.38862000e+07],
       [1.45350006e+02, 1.46660004e+02, 1.41160004e+02, 1.43210007e+02,
        8.22912000e+07],
       [1.41110001e+02, 1.41800003e+02, 1.36029999e+02, 1.36500000e+02,
        9.89319000e+07],
       [1.36690002e+02, 1.37649994e+02, 1.33729996e+02, 1.34509995e+02,
        1.60156900e+08],
       [1.35110001e+02, 1.35199997e+02, 1.31320007e+02, 1.32369995e+02,
        7.95926000e+07],
       [1.31389999e+02, 1.33250000e+02, 1.29889999e+02, 1.32300003e+02,
        7.74328000e+07],
       [1.32979996e+02, 1.36809998e+02, 1.32750000e+02, 1.35449997e+02,
        8.59280000e+07],
       [1.34350006e+02, 1.34559998e+02, 1.30300003e+02, 1.32229996e+02,
        7.78521000e+07],
       [1.30919998e+02, 1.32419998e+02, 1.29639999e+02, 1.31860001e+02,
        6.38149000e+07],
       [1.31380005e+02, 1.3141

In [57]:
y_pred= xgb_predictor.predict(df_array).decode('utf-8')
y_pred


'70.27188873291016\n70.27188873291016\n70.27188873291016\n70.27188873291016\n70.27188873291016\n70.27188873291016\n70.27188873291016\n70.27188873291016\n70.27188873291016\n70.27188873291016\n70.27188873291016\n70.27188873291016\n70.27188873291016\n70.27188873291016\n'

In [75]:
!git checkout develop


fatal: not a git repository (or any parent up to mount point /home/ec2-user)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
